In [2]:
!pip install pmdarima -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.1 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datetime import datetime
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams

from statsmodels.tsa.stattools import adfuller
import pmdarima as pm
from statsmodels.tsa.seasonal import seasonal_decompose

from pandas.tseries.holiday import USFederalHolidayCalendar
import itertools
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [5]:
def get_fourier_features(n_order, period, values):
    fourier_features = pd.DataFrame(
        {
            f"fourier_{func}_order_{order}": getattr(np, func)(
                2 * np.pi * values * order / period
            )
            for order in range(1, n_order + 1)
            for func in ("sin", "cos")
        }
    )
    return fourier_features

In [28]:
train = pd.read_csv('../input/demand-forecasting-kernels-only/train.csv', parse_dates = ['date'])
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [29]:
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2013-01-01', end='2018-03-31').to_pydatetime()

def is_holiday(value):
  if value in holidays:
    return 1
  return 0

In [30]:
train['is_hol'] = train['date'].apply(is_holiday)
train.head()

,date,store,item,sales,is_hol
0,2013-01-01,1,1,13,1
1,2013-01-02,1,1,11,0
2,2013-01-03,1,1,14,0
3,2013-01-04,1,1,13,0
4,2013-01-05,1,1,10,0


In [31]:
test = pd.read_csv('../input/demand-forecasting-kernels-only/test.csv', parse_dates = ['date'])
test.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


In [11]:
test['is_hol'] = test['date'].apply(is_holiday)
test.head()

,id,date,store,item,is_hol
0,0,2018-01-01,1,1,1
1,1,2018-01-02,1,1,0
2,2,2018-01-03,1,1,0
3,3,2018-01-04,1,1,0
4,4,2018-01-05,1,1,0


In [36]:
def predict(store, item):
  target = train.loc[(train.store == store) & (train.item == item)]
  print(target)
  target["t"] = np.arange(len(target))

  to_predict = test.loc[(test.store == store) & (test.item == item)]
  to_predict["t"] = np.arange(len(target), len(target) + len(to_predict))

  best_aicc = None
  best_norder = 4

  PERIOD = len(target)
  n_steps = len(to_predict)

  # for n_order in range(1, 4):
  #     train_fourier_features = get_fourier_features(n_order, PERIOD, target['t'] )
  #     arima_exog_model = pm.auto_arima(
  #         y=np.log(target["sales"]),
  #         exogenous=train_fourier_features,
  #         seasonal=False,
  #     )
  #     if best_aicc is None or arima_exog_model.aicc() < best_aicc:
  #         best_aicc = arima_exog_model.aicc()
  #         best_norder = n_order
  # print(f'Found best order {best_norder} with aicc of {best_aicc}')

  train_fourier_features = get_fourier_features(best_norder, PERIOD, target['t'])
  arima_exog_model = pm.auto_arima(
      y=target["sales"],
      exogenous=train_fourier_features,
      seasonal=False,
  )
  print('Finished training model...')
  test_fourier_features = get_fourier_features(best_norder, PERIOD, to_predict['t'])
  y_arima_exog_forecast = arima_exog_model.predict(
      n_periods=n_steps,
      exogenous=test_fourier_features,
  )
  to_predict["sales"] = y_arima_exog_forecast
  to_predict["sales"] = y_arima_exog_forecast.to_list()
  f = lambda x: int(x)
  to_predict["sales"] = to_predict["sales"].apply(f)
  return to_predict

In [39]:
results = pd.DataFrame()

for i in range(1, 51):
  for j in range(1, 11):
    print(f'Predicting for store {j} and item {i}')
    sub_results = predict(j, i)
    results = pd.concat([results, sub_results])

Predicting for store 1 and item 1
           date  store  item  sales  is_hol
0    2013-01-01      1     1     13       1
1    2013-01-02      1     1     11       0
2    2013-01-03      1     1     14       0
3    2013-01-04      1     1     13       0
4    2013-01-05      1     1     10       0
...         ...    ...   ...    ...     ...
1821 2017-12-27      1     1     14       0
1822 2017-12-28      1     1     19       0
1823 2017-12-29      1     1     15       0
1824 2017-12-30      1     1     27       0
1825 2017-12-31      1     1     23       0

[1826 rows x 5 columns]
Finished training model...
Predicting for store 2 and item 1
           date  store  item  sales  is_hol
1826 2013-01-01      2     1     12       1
1827 2013-01-02      2     1     16       0
1828 2013-01-03      2     1     16       0
1829 2013-01-04      2     1     20       0
1830 2013-01-05      2     1     16       0
...         ...    ...   ...    ...     ...
3647 2017-12-27      2     1     19       0


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Finished training model...
Predicting for store 4 and item 1
           date  store  item  sales  is_hol
5478 2013-01-01      4     1     10       1
5479 2013-01-02      4     1     12       0
5480 2013-01-03      4     1      8       0
5481 2013-01-04      4     1     15       0
5482 2013-01-05      4     1     19       0
...         ...    ...   ...    ...     ...
7299 2017-12-27      4     1     15       0
7300 2017-12-28      4     1     26       0
7301 2017-12-29      4     1     28       0
7302 2017-12-30      4     1     21       0
7303 2017-12-31      4     1     23       0

[1826 rows x 5 columns]


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Finished training model...
Predicting for store 5 and item 1
           date  store  item  sales  is_hol
7304 2013-01-01      5     1     11       1
7305 2013-01-02      5     1      9       0
7306 2013-01-03      5     1     12       0
7307 2013-01-04      5     1      8       0
7308 2013-01-05      5     1     13       0
...         ...    ...   ...    ...     ...
9125 2017-12-27      5     1     15       0
9126 2017-12-28      5     1     13       0
9127 2017-12-29      5     1     18       0
9128 2017-12-30      5     1     19       0
9129 2017-12-31      5     1     24       0

[1826 rows x 5 columns]


...

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Finished training model...
Predicting for store 10 and item 50
             date  store  item  sales  is_hol
911174 2013-01-01     10    50     33       1
911175 2013-01-02     10    50     37       0
911176 2013-01-03     10    50     46       0
911177 2013-01-04     10    50     51       0
911178 2013-01-05     10    50     41       0
...           ...    ...   ...    ...     ...
912995 2017-12-27     10    50     63       0
912996 2017-12-28     10    50     59       0
912997 2017-12-29     10    50     74       0
912998 2017-12-30     10    50     62       0
912999 2017-12-31     10    50     82       0

[1826 rows x 5 columns]


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Finished training model...


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [46]:
submission = results[['sales']]
submission.index.name = 'id'

,sales
id,
0,15
1,14
2,14
3,16
4,19
...,...
44995,55
44996,55
44997,62


In [47]:
submission.to_csv('submission.csv')